#### 交互式了解Aestas的使用方法

逐条执行下面的代码块，了解Aestas的使用方法。
在开始之前，确保.NET Interactive，pwsh已经安装

pwsh可以用包管理器安装。在Windows上，请考虑使用
```
winget install Microsoft.PowerShell
```

In [2]:
$aestasdoc = Get-Location
$aestas = Split-Path -Path $aestasdoc
$aestasdll = (Convert-Path ../bin/Release/net8.0).Replace("\", "/")

首先编译Aestas

In [ ]:
cd $aestas
./aestas.ps1 build

fsi并不支持相对路径，所以把获得的绝对路径保存到临时脚本中，让路径和这个notebook文件无关的同时还能加载Aestas

In [3]:
cd $aestasdoc
"#r `"$aestasdll/Aestas.Core.dll`"
#r `"$aestasdll/aestas.dll`"" | Out-File .load.fsx

In [ ]:
#r "nuget:FSharpPlus"
#load ".load.fsx"
open FSharpPlus
open Aestas
open Aestas.Prim
open Aestas.Core
open Aestas.Core.Builtin
open Aestas.Core.AestasBot
open Aestas.Core.CommandExecuter

创建Domain

*VirtualDomain*是一个虚拟的Domain，我们借助它的接口和它绑定的Bot来交互

In [5]:
let botName = "Vespera"
let chatHistory = arrList<string*(AestasContent list)>()
let virtualDomain = VirtualDomain((fun mid contents -> chatHistory.Add(botName, contents)),
    ignore, {
        uid = 1u
        name = botName
    },{
        uid = 0u
        name = "You"
    }, 0u, "Test", true)
let sendToBot x = 
    let contents = [AestasText x]
    chatHistory.Add("You", contents)
    match virtualDomain.Input contents |> Async.RunSynchronously with
    | Ok (), _ -> chatHistory[max (chatHistory.Count-2) 0..].DisplayTable() |> ignore
    | Error msg, _ -> printfn "Error: %s" msg

编写一个最简单的“语言模型”代码，让Bot能够重复它的输入

In [6]:
type Repeater() =
    interface ILanguageModelClient with
        member _.GetReply bot domain = 
            async {
                return chatHistory[^0] |> snd |> Ok, ignore
            }
        member _.CacheMessage bot domain message = ()
        member _.CacheContents bot domain contents = ()
        member _.ClearCache domain = ()
        member _.RemoveCache domain messageID = ()

创建一个Bot. 这次用的函数相比之前的教程更复杂，之后会解释参数的含义

In [8]:
let bot = createBot {|
    name = botName
    model = Repeater()
    systemInstruction = Some ""
    systemInstructionBuilder = None
    friendStrategy = None
    contentLoadStrategy = None
    contentParseStrategy = None
    messageReplyStrategy = Some StrategyReplyAll
    messageCacheStrategy = Some StrategyCacheAll
    contextStrategy = None
    inputPrefixBuilder = None
    userCommandPrivilege = None
|}
bindDomain bot virtualDomain

在之后可以执行下面的单元格清除所有上下文，以便重新开始

In [9]:
bot.ClearCachedContext virtualDomain
chatHistory.Clear()

在之后可以执行下面的单元格来显示上下文

In [ ]:
chatHistory.DisplayTable()

尝试发送第一条消息. 正常情况下，Bot会重复你的消息

In [ ]:
sendToBot "Hello"

为Bot添加指令系统

In [ ]:
addCommandExecuter bot "/" (makeExecuterWithBuiltinCommands [])
bot.CommandExecuters

尝试一条指令

In [ ]:
sendToBot "/version"

编写命令*add2*，把输入的两个数字相加并发送出来

In [ ]:
tryAddCommand bot.CommandExecuters["/"] {
    name = "add2"
    description = "Add two numbers"
    accessibleDomain = CommandAccessibleDomain.All
    privilege = CommandPrivilege.Normal
    execute = fun executer env args ->
        match args with
        | [|str0; str1|] ->
            let i0, i1 = trySscanf "%d" str0, trySscanf "%d" str1
            match i0, i1 with
            | Some i0, Some i1 ->
                i0 + i1 |> string |> env.log
            | _ ->  env.log "Invalid arguments"
        | _ -> env.log "Invalid arguments"
} |> printfn "%A"

尝试刚才编写的指令

In [ ]:
sendToBot "/add2 2 3"
sendToBot "/add2 2"

了解ContentParser. 首先，写一个会使用ContentParser的“语言模型”

In [17]:
type AlwaysBlank() =
    interface ILanguageModelClient with
        member _.GetReply bot domain =
            async {
                return "#[blank]" |> modelOutputParser bot domain |> Ok, ignore
            }
        member _.CacheMessage bot domain message = ()
        member _.CacheContents bot domain contents = ()
        member _.ClearCache domain = ()
        member _.RemoveCache domain messageID = ()

执行下面的代码，观察输出

In [ ]:
AlwaysBlank() |> bindModel bot
sendToBot "Hello"